### Load data

In [1]:
# coding: utf-8

# import spacy
# from spacy.lang.en import English 
# from spacy.tokens import Doc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os

In [2]:
metabolite_file = '../../Data/hmdb_mDivided.csv'
met_lib = pd.read_csv(metabolite_file)
print(met_lib.shape)

import json
syn_lib = json.load(open('../../Data/SYNONYMS.json'))
print(len(syn_lib))

(217920, 4)
217920


In [3]:
# Check at 391, 394 - nan synonyms
print(met_lib.iloc[0])
syn_lib[0]

accession                                           HMDB0000001
iupac_name    b'(2S)-2-amino-3-(1-methyl-1H-imidazol-4-yl)pr...
name                                       b'1-Methylhistidine'
synonym       (2S)-2-Amino-3-(1-methyl-1H-imidazol-4-yl)prop...
Name: 0, dtype: object


['(2S)-2-Amino-3-(1-methyl-1H-imidazol-4-yl)propanoic acid',
 'Pi-methylhistidine',
 '(2S)-2-Amino-3-(1-methyl-1H-imidazol-4-yl)propanoate',
 '1 Methylhistidine',
 '1-Methyl histidine',
 '1-Methyl-histidine',
 '1-Methyl-L-histidine',
 '1-MHis',
 '1-N-Methyl-L-histidine',
 'L-1-Methylhistidine',
 'N1-Methyl-L-histidine',
 '1-Methylhistidine dihydrochloride',
 '1-Methylhistidine']

### Preprocess terms and add to patterns file

#### Creating a pseudo-A list

In [13]:
names_processed = []
total_entries = 10000  # Change values and try which works (tried till 50k, can try more than that) # For full use len(met_lib)
start_time = time.time()

for i in range(total_entries):
## help
    # Clean text, remove 'b' from the beginning of string
    full_name = met_lib.iloc[i]['name'].strip("b\'\"")
    full_name_decoded = full_name.encode().decode() # \u2019 - ' urindine
    names_processed.append(full_name_decoded)
    
print('Metabolite names processed', len(names_processed))

A_list = set() # Set to keep unique values

# Select only one-word names, and these one-word names must have 5 to 10 characters
for i in range(len(names_processed)):
    if(len(names_processed[i]) >= 5 and len(names_processed[i]) <= 11 and len(names_processed[i].split()) == 1): 
        ''' Can 10 Oleic acid and 11 Adipic acid be considered even though they have two words? - No '''
        A_list.add(names_processed[i])

print('Number of names in A_list ', len(A_list))
print('Time taken to prepare A list ', round(time.time() - start_time, 4))

Metabolite names processed 10000
Number of names in A_list  1404
Time taken to prepare A list  0.6823


In [11]:
A_list = list(A_list)

In [7]:
# print('Adenosine monophosphate' in A_list)
# print('monophosphate' in A_list)
# print('Adenosine' in A_list)


#### Create a new_metabolite_lib that contains only those terms that has at least one occurence of the names in A_list

In [151]:
start_time = time.time()
new_met_lib_index = []

# Need to remove that b character (represents bytes but it is string here in df) from iupac_name and name
metabolites_patterns = []


verbose_frequency1=400
verbose_frequency2=400
nn1,nn2=0,0
for ii in range(len(names_processed)):
    
    ## process name column
    flag_i1=False
    # Check if word is acceptable - to be acceptable it should have some occurence of any word from A_list
## but names_processed[ii] is a string ?
    # if( any(x in names_processed[ii] for x in A_list) ):
    if any(x.lower() in names_processed[ii].lower() for x in A_list):
        #print(names_processed[ii])
        # Add to patterns file
        flag_i1=True
        id_name = names_processed[ii]
        
        # Have the original name as a pattern for exact match of main name, so process the word also
## change to normal
        full_name_spaced = ''.join((' {} '.format(el.lower()) if not el.isalnum() and not el.isspace() else el for el in names_processed[ii]))
        full_name_list = full_name_spaced.split()
        add_split_pattern = []
    
        for split_word in full_name_list:
            add_split_pattern.append({"LOWER": str(split_word.lower())}) # The first lower did not work
        
        metabolites_patterns.append({"label": "Metabolites", "pattern": add_split_pattern, "id": id_name })
        
    if flag_i1:
        nn1+=1
        if verbose_frequency1 and nn1%verbose_frequency1==0:
            for x in A_list:
                if x.lower() in names_processed[ii].lower():
                    print("name col:",nn1,ii,"::",names_processed[ii],"::",x)
                    print(metabolites_patterns[-1])
                    print()
                    break
    
    ## process synonyms column
    
    flag_i2=False
    syn_spaced2=''
    x2=''
    # There are many metabolites with nan synonyms, so ignore them
    if(syn_lib[ii] != ['nan'] and syn_lib[ii] != [''] and syn_lib[ii] != [' '] ):

        # No Need to split synonym
        syn_list = syn_lib[ii]
        #if ii==0: print(syn_list)
        #print()
        
        # Fixing issue of synonyms being just numbers. 
        # Solution is to filter and check if a name contains only numbers (isnumeric), if so then do not include in patterns file.
## change to normal
        syn_list = [item for item in syn_list if not item.isnumeric() and len(item) > 1] # This greater than 1 condition to avoid single alphabets
## why single alpha ?
        # if ii==0:
        #     print(syn_list)
            
        for syn in syn_list:
            # Doing encode.decode even on normal strings does not affect anything, but helps with strings having unicode characters
            syn_decoded = syn.encode().decode()
## change to normal
            syn_spaced = ''.join((' {} '.format(el.lower()) if not el.isalnum() and not el.isspace() else el for el in syn_decoded))
            #if ii==0: print(syn_spaced)
            syn_split = syn_spaced.split()
            
            ##check whether A_list in current syn
            flag=False
            
#             for x in A_list:
#                 if x.lower() in syn_spaced.lower():
#                     flag_i=True
#                     syn_spaced2=syn_spaced
#                     x2=x
#                     # print(ii,syn_spaced,":",x)
            
##issue2
            if any(x.lower() in syn_spaced.lower() for x in A_list):
                flag=True
                flag_i2=True
                syn_spaced2=syn_spaced
                # if ii==42: 
                #     print(syn_spaced)
                #     print()
            # if flag: print(ii,syn_spaced,":",x)
                    
            if flag:
                # If two letter word then do case sensitive exact match. This is to handle AS, AT words, not to be confused with as, at.
                if(len(syn_spaced) == 2):
                    metabolites_patterns.append({"label": "Metabolites", "pattern": syn_spaced, "id": id_name })
                else:  
                    # Having double spaces also does not affect as .split() removes any word between any number of spaces
                    add_split_pattern = []
                    for split_word in syn_split:
                        add_split_pattern.append({"LOWER": str(split_word.lower())}) # The first lower did not work
                    metabolites_patterns.append({"label": "Metabolites", "pattern": add_split_pattern, "id": id_name })
    
    if flag_i2:
        nn2+=1
        if verbose_frequency2 and nn2%verbose_frequency2==0:
            for x in A_list:
                if x.lower() in syn_spaced2.lower():
                    print("syn col:",nn2,ii,"::",syn_spaced2,"::",x)
                    print(metabolites_patterns[-1])
                    print()
                    break

syn col: 400 686 :: Ergosterol :: Ergosterol
{'label': 'Metabolites', 'pattern': [{'LOWER': 'ergosterol'}], 'id': 'Ergosterol'}

name col: 400 965 :: Methylmalonyl-CoA :: Malonyl-CoA
{'label': 'Metabolites', 'pattern': [{'LOWER': 'methylmalonyl'}, {'LOWER': '-'}, {'LOWER': 'coa'}], 'id': 'Methylmalonyl-CoA'}

syn col: 800 1265 :: Ibuprofen ,  sodium salt :: Sodium
{'label': 'Metabolites', 'pattern': [{'LOWER': 'ibuprofen'}, {'LOWER': ','}, {'LOWER': 'sodium'}, {'LOWER': 'salt'}], 'id': 'Ibuprofen'}

name col: 800 1801 :: 1-Methylguanine :: Guanine
{'label': 'Metabolites', 'pattern': [{'LOWER': '1'}, {'LOWER': '-'}, {'LOWER': 'methylguanine'}], 'id': '1-Methylguanine'}

syn col: 1200 2010 :: a , 4 - Dihydroxy - 3 - methoxy - benzeneacetaldehyde :: Benzene
{'label': 'Metabolites', 'pattern': [{'LOWER': 'a'}, {'LOWER': ','}, {'LOWER': '4'}, {'LOWER': '-'}, {'LOWER': 'dihydroxy'}, {'LOWER': '-'}, {'LOWER': '3'}, {'LOWER': '-'}, {'LOWER': 'methoxy'}, {'LOWER': '-'}, {'LOWER': 'benzeneacetal

In [144]:
syn_lib[42]

['3\'-Phosphoadenylate", "Adenosine 3\',5\'-bisphosphate',
 'PAP',
 'Phosphoadenosine phosphate',
 '3\'-Phosphoadenylic acid", "Adenosine 3\',5\'-bisphosphoric acid',
 'Phosphoadenosine phosphoric acid',
 'Adenosine 3\',5\'-diphosphoric acid", "3\'-Phosphoryl-AMP',
 '3,5-ADP',
 '3,5-Diphosphoadenosine',
 '3-Phosphoadenosine 5-phosphate',
 '5-(Dihydrogen phosphate) 3-adenylate',
 "5-(Dihydrogen phosphate)3'-adenylic acid",
 'Adenosine 3,5-bis',
 'Adenosine 3,5-bisphosphate',
 'Adenosine 3\'-phosphate-5\'-phosphate, disodium salt", "3\'-Phosphoadenosine 5\'-phosphate", "Adenosine 3\'-phosphate-5\'-phosphate, monosodium salt", "3\',5\'-ADP", "Adenosine 3\'-phosphate-5\'-phosphate']

In [143]:
syn_lib[42][-1]

'Adenosine 3\'-phosphate-5\'-phosphate, disodium salt", "3\'-Phosphoadenosine 5\'-phosphate", "Adenosine 3\'-phosphate-5\'-phosphate, monosodium salt", "3\',5\'-ADP", "Adenosine 3\'-phosphate-5\'-phosphate'

In [131]:
metabolites_patterns[:3]

[{'label': 'Metabolites',
  'pattern': [{'LOWER': 'alpha'},
   {'LOWER': '-'},
   {'LOWER': 'ketobutyric'},
   {'LOWER': 'acid'},
   {'LOWER': ','},
   {'LOWER': 'sodium'},
   {'LOWER': 'salt'}],
  'id': 'Hydroxyacetone'},
 {'label': 'Metabolites',
  'pattern': [{'LOWER': '2'}, {'LOWER': '-'}, {'LOWER': 'methoxyestrone'}],
  'id': '2-Methoxyestrone'},
 {'label': 'Metabolites',
  'pattern': [{'LOWER': '2'},
   {'LOWER': '-'},
   {'LOWER': 'hydroxyestrone'},
   {'LOWER': '2'},
   {'LOWER': '-'},
   {'LOWER': 'methyl'},
   {'LOWER': 'ether'}],
  'id': '2-Methoxyestrone'}]

In [132]:
len(metabolites_patterns)

2550

### Now for abstracts: For every keyword in A_list, extract 1000 abstracts per keyword. There should be presence of that word in that abstract anywhere - not just title.
### And then do pattern-matching using our patterns file

### Below is my scraping method - not very efficient

In [14]:
# import csv
# import re
# import urllib
# from time import sleep

# query = "Cortexolone"

# # common settings between esearch and efetch
# base_url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
# db = 'db=pubmed'

# # esearch settings
# search_eutil = 'esearch.fcgi?'
# search_term = '&term=' + query
# search_usehistory = '&usehistory=y'
# search_rettype = '&rettype=json'

# # call the esearch command for the query and read the web result
# search_url = base_url+search_eutil+db+search_term+search_usehistory+search_rettype
# print("this is the esearch command:\n" + search_url + "\n")
# f = urllib.request.urlopen (search_url)
# search_data = f.read().decode('utf-8')

# # extract the total abstract count
# total_abstract_count = int(re.findall("<Count>(\d+?)</Count>",search_data)[0])

# if(total_abstract_count < 10):
#     max_count = total_abstract_count
# else:
#     max_count = 10

# # efetch settings
# fetch_eutil = 'efetch.fcgi?'
# retmax = 20
# retstart = 0
# fetch_retmode = "&retmode=text"
# fetch_rettype = "&rettype=abstract"

# # obtain webenv and querykey settings from the esearch results
# fetch_webenv = "&WebEnv=" + re.findall ("<WebEnv>(\S+)<\/WebEnv>", search_data)[0]
# fetch_querykey = "&query_key=" + re.findall("<QueryKey>(\d+?)</QueryKey>",search_data)[0]

# # call efetch commands using a loop until all abstracts are obtained
# run = True
# all_abstracts = list()
# loop_counter = 1

# while len(all_abstracts) < max_count:
#     print("this is efetch run number " + str(loop_counter))
#     loop_counter += 1
#     fetch_retstart = "&retstart=" + str(retstart)
#     fetch_retmax = "&retmax=" + str(retmax)
#     # create the efetch url
#     fetch_url = base_url+fetch_eutil+db+fetch_querykey+fetch_webenv+fetch_retstart+fetch_retmax+fetch_retmode+fetch_rettype
#     print(fetch_url)
#     # open the efetch url
#     f = urllib.request.urlopen (fetch_url)
#     fetch_data = f.read().decode('utf-8')
#     # split the data into individual abstracts
#     abstracts = fetch_data.split("\n\n\n")
#     # append to the list all_abstracts
#     all_abstracts = all_abstracts+abstracts
#     print("a total of " + str(len(all_abstracts)) + " abstracts have been downloaded.\n")
#     # wait 2 seconds so we don't get blocked
#     sleep(2)
#     # update retstart to download the next chunk of abstracts
#     retstart = retstart + retmax
#     if retstart > total_abstract_count:
#         run = False

### Below is code found online for scraping - works fast but needs fixing

In [12]:
import gc

start_time = time.time()
abstract_writer = pd.DataFrame(columns = ['keyword', 'abstract'])
start_index = 0
end_index = len(A_list)

for i in range(start_index, end_index):
    
    if(i%100 == 0 and i!= 0): 
        time.sleep(100)
        
    time.sleep(2)
    
    gc.collect()
    # to get abstracts
    query_name = A_list[i]
    fc = 0
    with open('keywords.txt', 'w') as f:
        f.write(query_name)
        fc = 1
    f.close()
    
    
    '''
    !python3 async_pubmed_scraper_2010_2015.py --pages=10 --start=2020 --end=2022 --output='article_data.csv' # Separating out years because it can not do 2000 to 2022 at a time (gives error) 2017 - 2022

    '''

    try:
        # Round 1
        !python3 async_pubmed_scraper.py # Separating out years because it can not do 2000 to 2022 at a time (gives error) 2017 - 2022

        # Once output file is created, parse it and get necessary values
        if(fc == 1):
            file = 'article_data.csv'
            abs_pbs = pd.read_csv(file)

            abstracts = abs_pbs['abstract']
            print('Number of abstracts for', query_name, ' is ', len(abstracts))

            for j in range(len(abstracts)):
                text = str(abstracts.iloc[j])
                if(text and text != 'NO_ABSTRACT' and str(text) != 'nan'):
                    abstract_writer = abstract_writer.append({'keyword': query_name, 'abstract': text}, ignore_index=True)

    except Exception as e: 
        print('error at ', i, ' with exception ')
        
print('Time taken to get abstracts ', round(time.time() - start_time, 4))



Finding PubMed article URLs for 1 keywords found in keywords.txt

Scraping initiated for 100 article URLs found from 2017 to 2021

It took 12.25301456451416 seconds to find 100 articles; 100 unique articles were saved to article_data.csv
Number of abstracts for Acetoin  is  100

Finding PubMed article URLs for 1 keywords found in keywords.txt

Scraping initiated for 100 article URLs found from 2017 to 2021

It took 11.352988958358765 seconds to find 100 articles; 100 unique articles were saved to article_data.csv
Number of abstracts for Baclofen  is  100

Finding PubMed article URLs for 1 keywords found in keywords.txt

Scraping initiated for 71 article URLs found from 2017 to 2021

It took 8.848851203918457 seconds to find 71 articles; 69 unique articles were saved to article_data.csv
Number of abstracts for Picrocrocin  is  69

Finding PubMed article URLs for 1 keywords found in keywords.txt

Scraping initiated for 36 article URLs found from 2017 to 2021

It took 4.2440125942230225 

In [ ]:
# Now all we have to do is do NER pattern matching to get entities and then for each of those entites, check if they
# are present in the A list, if they are there then record it.

### Very important note: Save A_list also, and use that A_list to accept or reject entity names from NER. For some reason, A_list differs everytime I run - check why.

In [15]:
A_list_df = pd.DataFrame(data = A_list, columns = ['A_list_names'])
A_list_df.to_csv('A_list_31_01.csv', index=False)

In [16]:
abstract_writer.to_csv('abstract_writer_31_01.csv', index=False)

In [17]:
abstract_writer.shape

(98901, 2)

In [19]:
abstract_writer['keyword'].value_counts()

Acetoin        100
CE(22:0)       100
Sulindac       100
Carnosol       100
Ticarcillin    100
              ... 
12-KETE          1
Hawkinsin        1
Diflorasone      1
L-Canaline       1
Spirapril        1
Name: keyword, Length: 1430, dtype: int64

In [20]:
abstract_writer.head()

,keyword,abstract
0,Acetoin,Serratia marcescens is reported to possess the...
1,Acetoin,Acetoin (3-hydroxy-2-butanone) is an important...
2,Acetoin,A set of 917 wines of Czech origin were analys...
3,Acetoin,Microbial production of acetoin is eco-friendl...
4,Acetoin,"In this study, physicochemical composition, ni..."


In [29]:
al = list(map(str.lower, A_list))

In [27]:
'Acetoin' in A_list

True

In [32]:
'acetoin' in A_list

False

In [ ]:
import csv, time 
from nltk import tokenize

start_index = 0
end_index = len(A_list)
document_index = 0

# Do in separate loops
abstract_writer = pd.DataFrame(columns = ['keyword', 'abstract'])

for i in range(start_index, end_index): 

    if(i%100 == 0 and i!= 0): 
        time.sleep(180)
        print(i) 

    # to get abstracts
    disease_name = A_list[i]
    fc = 0
    with open('keywords.txt', 'w') as f:
        f.write(disease_name)
        fc = 1

    # run the pubmed scraper
    try:

        !python3 async_pubmed_scraper.py

        # Once output file is created, parse it and get necessary values
        if(fc == 1):

            file = 'article_data.csv'
            abs_pbs = pd.read_csv(file)

            # now run the rev_irrev function
            outputs = []
            abstracts = abs_pbs['abstract']
            print('Number of abstracts for ', disease_name, ' is ', len(abstracts))

            for j in range(len(abstracts)):
                text = str(abstracts.iloc[j])
                if(text and text != 'NO_ABSTRACT'):
                    print('done')
                    abstract_writer = abstract_writer.append({'keyword': disease_name, 'abstract': text}, ignore_index=True)

    except Exception as e: 
        print('error at ', i, ' with exception ', str(e))

out_file.close()



Finding PubMed article URLs for 1 keywords found in keywords.txt

Scraping initiated for 100 article URLs found from 2017 to 2021

Traceback (most recent call last):
  File "async_pubmed_scraper.py", line 289, in <module>
  File "/N/u/paswam/Carbonate/.local/lib/python3.6/site-packages/pandas/core/generic.py", line 3167, in to_csv
  File "/N/u/paswam/Carbonate/.local/lib/python3.6/site-packages/pandas/io/formats/csvs.py", line 190, in save
  File "/N/u/paswam/Carbonate/.local/lib/python3.6/site-packages/pandas/io/common.py", line 493, in get_handle
MemoryError
error at  0  with exception  No columns to parse from file

Finding PubMed article URLs for 1 keywords found in keywords.txt

Scraping initiated for 100 article URLs found from 2017 to 2021

Fatal read error on socket transport
protocol: <asyncio.sslproto.SSLProtocol object at 0x2aca84a99128>
transport: <_SelectorSocketTransport fd=79 read=polling write=<idle, bufsize=0>>
Traceback (most recent call last):
  File "/usr/lib64/pyt

In [ ]:

for keyword_name in A_list:
    gc.collect()
    # to get abstracts
    query_name = keyword_name
    fc = 0
    with open('keywords.txt', 'w') as f:
        f.write(query_name)
        fc = 1
    f.close()

    time.sleep(10)

    try:

        time.sleep(5)

        !python3 async_pubmed_scraper_2005_2010.py

        # Once output file is created, parse it and get necessary values
        if(fc == 1):
            file = 'article_data.csv'
            abs_pbs = pd.read_csv(file)

            abstracts = abs_pbs['abstract']
            print('Number of abstracts for ', query_name, ' is ', len(abstracts))

            for j in range(len(abstracts)):
                abstract_writer = abstract_writer.append({'keyword': query_name, 'abstract': str(abstracts.iloc[j])}, ignore_index=True)


        time.sleep(5)

        !python3 async_pubmed_scraper_2000_2005.py

        # Once output file is created, parse it and get necessary values
        if(fc == 1):
            file = 'article_data.csv'
            abs_pbs = pd.read_csv(file)

            abstracts = abs_pbs['abstract']
            print('Number of abstracts for ', query_name, ' is ', len(abstracts))

            for j in range(len(abstracts)):
                abstract_writer = abstract_writer.append({'keyword': query_name, 'abstract': str(abstracts.iloc[j])}, ignore_index=True)


    except Exception as e: 
        print('error at ', i, ' with exception ', str(e))


In [29]:
abstract_writer

""


In [142]:
abstract_csv = pd.read_csv('abstracts.csv', sep='delimiter', header=1)

/N/u/paswam/Carbonate/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
abstracts

["1. Clascoterone.\n\nDrugs and Lactation Database (LactMed®) [Internet]. Bethesda (MD): National \nInstitute of Child Health and Human Development; 2006–.\n2023 Jan 15.\n\nNo information is available on the clinical use of clascoterone during \nbreastfeeding. However, because it is poorly absorbed from the skin and 84% to \n89% protein bound, amounts in breastmilk are probably low and would not be \nexpected to cause any adverse effects in breastfed infants. Avoid application to \nthe nipple area and ensure that the infant's skin does not come into direct \ncontact with the areas of skin that have been treated.\n\nPMID: 36701501",
 '2. Front Endocrinol (Lausanne). 2022 Nov 10;13:1004129. doi: \n10.3389/fendo.2022.1004129. eCollection 2022.\n\nEvaluation of plasma ACTH in the metyrapone test is insufficient for the \ndiagnosis of secondary adrenal insufficiency.\n\nPapierska L(1), Rabijewski M(2), Migda B(3), Leszczyńska D(4), Nowak K(1), \nŁebek-Szatańska A(1), Glinicki P(1), Zgliczyń

In [ ]:

# Need to remove that b character (represents bytes but it is string here in df) from iupac_name and name
metabolites_patterns = []
record_per_word_times = []
total_entries = 1000  # Change values and try which works (tried till 50k, can try more than that) # For full use len(met_lib)

# For each value in metabolite library, decode it and then add to patterns 
  # While adding to patterns, aplit the full word on all special characters and each word put as {LOWER: that_word} for every full word
total_words_start_time = time.time()
for i in range(total_entries):
    try:
        start_time = time.time()
        # Clean text, remove 'b' from the beginning of string
        full_name = met_lib.iloc[i]['name'].strip("b\'\"")
        full_name_decoded = full_name.encode().decode() # \u2019 - ' urindine
        id_name = full_name_decoded
        
        # Have the original name as a pattern for exact match of main name, so process the word also
        full_name_spaced = ''.join((' {} '.format(el.lower()) if not el.isalnum() and not el.isspace() else el for el in full_name_decoded))
        full_name_list = full_name_spaced.split()
        add_split_pattern = []
    
        for split_word in full_name_list:
            add_split_pattern.append({"LOWER": str(split_word.lower())}) # The first lower did not work
        
        metabolites_patterns.append({"label": "Metabolites", "pattern": add_split_pattern, "id": id_name })

        # IUPAC Names has \xc2\\xb2,\\xe2\\x81\\xb7.0\\xc2\\xb9\\xc2\\xb9,\\xc2\\xb9\\x characters
        # Check if this is correct format
        # iupac = met_lib.iloc[i]['iupac_name'].strip("b\'\"")
        
        # There are many metabolites with nan synonyms, so ignore them
        if(str(met_lib.iloc[i]['synonym']) != 'nan' ):
            
            # Need to split synonym - delim is : character
            syn_list = met_lib.iloc[i]['synonym'].split(':')
            
            # Fixing issue of synonyms being just numbers. 
            # Solution is to filter and check if a name contains only numbers (isnumeric), if so then do not include in patterns file.
            syn_list = [item for item in syn_list if not item.isnumeric()] # This greater than 1 condition to avoid single alphabets
            for syn in syn_list:
                # Doing encode.decode even on normal strings does not affect anything, but helps with strings having unicode characters
                syn_decoded = syn.encode().decode()
                syn_spaced = ''.join((' {} '.format(el.lower()) if not el.isalnum() and not el.isspace() else el for el in syn_decoded))

                # Having double spaces also does not affect as .split() removes any word between any number of spaces
                syn_list = syn_spaced.split()
                add_split_pattern = []
            
                for split_word in syn_list:
                    add_split_pattern.append({"LOWER": str(split_word.lower())}) # The first lower did not work
                # if((add_split_pattern[0]['LOWER'] == "as") and (len(add_split_pattern)==1) ):
                #     print('werid', add_split_pattern, met_lib.iloc[i]['synonym'], full_name)

                metabolites_patterns.append({"label": "Metabolites", "pattern": add_split_pattern, "id": id_name })

        record_per_word_times.append(round(time.time() - start_time, 5))
    except Exception as e:
        print('Exception at', i,  'as ', e)

total_words_end_time = round(time.time() - total_words_start_time, 5)  
print('Total entries in Metabolite Patterns file ', len(metabolites_patterns)) 


### Add ruler to pipeline

In [ ]:
nlp = None
nlp = English()
ruler = nlp.add_pipe("entity_ruler", config={"phrase_matcher_attr": "LOWER"}) #can only work for non-token pattern
record_ruler_add_times = 0
start_time = time.time()
with nlp.select_pipes(enable="tagger"):
    ruler.add_patterns(metabolites_patterns)
record_ruler_add_times = round(time.time() - start_time, 5)
# print("--- %s seconds ---" % (time.time() - start_time))

# ruler.to_disk("patterns_spaced.jsonl")
# ruler.from_disk("patterns_spaced.jsonl")
print('Adding to ruler done')


### Now run the entity ruler matching algorithm on abstracts

In [ ]:

# Now we have the pattern ruler ready, run on one abstract
# Now preprocessing on abstracts (including unicode character removal) - IS THIS STEP NECESSARY BECAUSE IT SEEMS LIKE ABSTRACTS ALREADY HAVE SPACED WORDS (NOT SURE 
# IF THIS APPLIES TO ALL ABSTRACTS )
record_per_abstract_times = []
record_per_abstract_length = []

# What is the basis we choose abstracts - do we take on of A list?

sample_abstracts = ["Uridine 5 ' - diphosphate ( UDP ) and uridINe 5' - dipHOSphaTe - glucose dehydrogenase ( UGD ) produces UDP - glucuronic acid from UDP - glucose as a precursor of plant cell wall polysaccharides .UDP - glucuronic acid is also a sugar donor for the glycosylation of various plant specialized metabolites .Nevertheless , the roles of UGDs in plant specialized metabolism remain poorly understood .Glycyrrhiza species ( licorice ) , which are medicinal legumes , biosynthesize triterpenoid saponins , soyasaponins and glycyrrhizin , commonly glucuronosylated at the C - 3 position of the triterpenoid scaffold .",
"Uridine diphosphate glycosyltransferases (UGTs) are the key enzymes in glycosylation processes for decorating plant natural products with sugars. Crystallography, one of the powerful techniques for determining protein structures, was used as the main experimental technique and combined with biochemical methods to study the structure-function relationship and molecular mechanisms of UGTs. Crystal structures of plant UGTs have revealed their exquisite architectures and provided the structural basis for understanding their catalytic mechanism and substrate specificity. In this chapter, some protocols and experimental details of all key stages of protein structure determination are provided, and the structural insights on plant UGTs are also highlighted in combination of method description.",
"Lung cancer (LC) is the second most common cause of death in men after prostate cancer, and the third most recurrent type of tumor in women after breast and colon cancers. Unfortunately, when LC symptoms begin to appear, the disease is already in an advanced stage and the survival rate only reaches 2%. Thus, there is an urgent need for early diagnosis of LC using specific biomarkers, as well as effective therapies and strategies against LC. On the other hand, the influence of metals on more than 50% of proteins is responsible for their catalytic properties or structure, and their presence in molecules is determined in many cases by the genome. Research has shown that redox metal dysregulation could be the basis for the onset and progression of LC disease. Moreover, metals can interact between them through antagonistic, synergistic and competitive mechanisms, and for this reason metals ratios and correlations in LC should be explored. One of the most studied antagonists against the toxic action of metals is selenium, which plays key roles in medicine, especially related to selenoproteins. The study of potential biomarkers able to diagnose the disease in early stage is conditioned by the development of new analytical methodologies. In this sense, omic methodologies like metallomics, proteomics and metabolomics can greatly assist in the discovery of biomarkers for LC early diagnosis.",
"Omega-3 polyunsaturated fatty acid (ω-3 PUFA) supplements for chemoprevention of different types of cancer including lung cancer has been investigated in recent years. ω-3 PUFAs are considered immunonutrients, commonly used in the nutritional therapy of cancer patients. ω-3 PUFAs play essential roles in cell signaling and in cell structure and fluidity of membranes. They participate in the resolution of inflammation and have anti-inflammatory effects. Lung cancer patients suffer complications, such as anorexia-cachexia syndrome, pain and depression. The European Society for Clinical Nutrition and Metabolism (ESPEN) 2017 guidelines for cancer patients only discuss the use of ω-3 PUFAs for cancer-cachexia treatment, leaving aside other cancer-related complications that could potentially be managed by ω-3 PUFAs. This review aims to elucidate whether the effects of ω-3 PUFAs in lung cancer is supplementary, pharmacological or both. In addition, clinical studies, evidence in cell lines and animal models suggest how ω-3 PUFAs induce anticancer effects. ω-3 PUFAs and their metabolites are suggested to modulate pivotal pathways underlying the progression or complications of lung cancer, indicating that this is a promising field to be explored. Further investigation is still required to analyze the benefits of ω-3 PUFAs as supplementation or pharmacological treatment in lung cancer.",
"A causal association has been established between alcohol consumption and cancers of the oral cavity, pharynx, larynx, oesophagus, liver, colon, rectum, and, in women, breast; an association is suspected for cancers of the pancreas and lung. Evidence suggests that the effect of alcohol is modulated by polymorphisms in genes encoding enzymes for ethanol metabolism (eg, alcohol dehydrogenases, aldehyde dehydrogenases, and cytochrome P450 2E1), folate metabolism, and DNA repair. The mechanisms by which alcohol consumption exerts its carcinogenic effect have not been defined fully, although plausible events include: a genotoxic effect of acetaldehyde, the main metabolite of ethanol; increased oestrogen concentration, which is important for breast carcinogenesis; a role as solvent for tobacco carcinogens; production of reactive oxygen species and nitrogen species; and changes in folate metabolism. Alcohol consumption is increasing in many countries and is an important cause of cancer worldwide.",
"Ethanol is neither genotoxic nor mutagenic. Its first metabolite acetaldehyde, however, is a powerful local carcinogen. Point mutation in ALDH2 gene proves the causal relationship between acetaldehyde and upper digestive tract cancer in humans. Salivary acetaldehyde concentration and exposure time are the two major and quantifiable factors regulating the degree of local acetaldehyde exposure in the ideal target organ, oropharynx. Instant microbial acetaldehyde formation from alcohol represents >70% of total ethanol associated acetaldehyde exposure in the mouth. In the oropharynx and achlorhydric stomach acetaldehyde is not metabolized to safe products, instead in the presence of alcohol it accumulates in saliva and gastric juice in mutagenic concentrations. A common denominator in alcohol, tobacco and food associated upper digestive tract carcinogenesis is acetaldehyde. Epidemiological studies on upper GI tract cancer are biased, since they miss information on acetaldehyde exposure derived from alcohol and acetaldehyde present in 'non-alcoholic' beverages and food.",
"Infertility is a severe medical problem and is considered a serious global public health issue affecting a large proportion of humanity. Oxidative stress is one of the most crucial factors involved in infertility. Recent studies indicate that the overproduction of reactive oxygen species (ROS) or reactive nitrogen species (RNS) may cause damage to the male and female reproductive systems leading to infertility. Low amounts of ROS and RNS are essential for the normal functioning of the male and female reproductive systems, such as sperm motility, acrosome reaction, interactions with oocytes, ovulation, and the maturation of follicles. Environmental factors such as heavy metals can cause reproductive dysfunction in men and women through the overproduction of ROS and RNS. It is suggested that oxidative stress caused by arsenic is associated with male and female reproductive disorders such as through the alteration in sperm counts and motility, decreased sex hormones, dysfunction of the testis and ovary, as well as damage to the processes of spermatogenesis and oogenesis. This review paper highlights the relationship between arsenic-induced oxidative stress and the prevalence of infertility, with detailed explanations of potential underlying mechanisms.",
"The microbiota-gut-brain axis is a bidirectional signaling mechanism between the gastrointestinal tract and the central nervous system. The complexity of the intestinal ecosystem is extraordinary; it comprises more than 100 trillion microbial cells that inhabit the small and large intestine, and this interaction between microbiota and intestinal epithelium can cause physiological changes in the brain and influence mood and behavior. Currently, there has been an emphasis on how such interactions affect mental health. Evidence indicates that intestinal microbiota are involved in neurological and psychiatric disorders. This review covers evidence for the influence of gut microbiota on the brain and behavior in Alzheimer disease, dementia, anxiety, autism spectrum disorder, bipolar disorder, major depressive disorder, Parkinson's disease, and schizophrenia. The primary focus is on the pathways involved in intestinal metabolites of microbial origin, including short-chain fatty acids, tryptophan metabolites, and bacterial components that can activate the host's immune system. We also list clinical evidence regarding prebiotics, probiotics, and fecal microbiota transplantation as adjuvant therapies for neuropsychiatric disorders.",
"O - linked N - acetylglucosamine ( O - GlcNAc ) is a dynamic post - translational modification occurring on myriad proteins in the cell nucleus , cytoplasm , and mitochondria .The donor sugar for O - GlcNAcylation , uridine - diphosphate N - acetylglucosamine ( UDP - GlcNAc ) , is synthesized from glucose through the hexosamine biosynthetic pathway ( HBP ) .",
"Cholesterol is a multifaceted metabolite that is known to modulate processes in cancer, atherosclerosis, and autoimmunity. A common denominator between these diseases appears to be the immune system, in which many cholesterol-associated metabolites impact both adaptive and innate immunity. Many cancers display altered cholesterol metabolism, and recent studies demonstrate that manipulating systemic cholesterol metabolism may be useful in improving immunotherapy responses. However, cholesterol can have both proinflammatory and anti-inflammatory roles in mammals, acting via multiple immune cell types, and depending on context. Gaining mechanistic insights into various cholesterol-related metabolites can improve our understanding of their functions and extensive effects on the immune system, and ideally will inform the design of future therapeutic strategies against cancer and/or other pathologies.",
"The recycling of O - GlcNAc on proteins is mediated by two enzymes in cells - O - GlcNAc transferase ( OGT ) and O - GlcNAcase ( OGA ) , which catalyze the addition and removal of O - GlcNAc , respectively .O - GlcNAcylation is involved in a number of important cell processes including transcription , translation , metabolism , signal transduction , and apoptosis .Deregulation of O - GlcNAcylation has been reported to be associated with various human diseases such as cancer , diabetes , neurodegenerative diseases , and cardiovascular diseases .(0 - not any proper association)"]
# new_abstract = []

for abstract in sample_abstracts:
    start_time = time.time()
    abstract_spaced = "".join((' {} '.format(el.encode().decode()) if (not el.isalnum() and not el.isspace()) else el for el in abstract))
    abstract_preporcessed = " ".join(abstract_spaced.split())
    # new_abstract.append(abstract_preporcessed)

    ''' For list of documents (DO THIS OUTSIDE LOOP)
    docs = list(nlp.pipe(new_abstract))
    c_doc = Doc.from_docs(docs)
    doc = nlp(c_doc)
    '''

    ''' For single document at a time
    doc = nlp(abstract)
    '''

    doc = nlp(abstract_preporcessed)

    #print NER result
    print('\n')
    print('Next abstract results:')
    print([(ent.text, ent.label_, ent.ent_id_) for ent in doc.ents])
    record_per_abstract_times.append(round(time.time() - start_time, 5))
    record_per_abstract_length.append(len(abstract))
    # print("--- %s seconds ---" % (time.time() - start_time))

